# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [37]:
import pandas as pd
import numpy as np

data_set = pd.read_csv("../data/bank_transactions_reduced.csv")
data_set.sample(5)

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
781,PAYMENT,12055.15,C1326769257,21356.0,9300.85,M1422139720,0.00,0.00,0,0
3269,PAYMENT,5307.07,C1615352805,15942.0,10634.93,M1520794599,0.00,0.00,0,0
4866,TRANSFER,308375.42,C265021960,90477.0,0.00,C2080794562,1546813.37,1855188.79,0,0
2405,CASH_IN,36569.81,C1656675734,77079.0,113648.81,C728922327,0.00,0.00,0,0
5273,CASH_OUT,118794.17,C1762654121,0.0,0.00,C1824444920,1393645.93,1512440.11,0,0


## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

Answer here

- The non-predictive columns are `nameOrig`, `nameDest`, `isFlaggedFraud`l. We can use `df.drop()` to remove these columns.

In [38]:
data_set = data_set.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'])
data_set.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,CASH_IN,63516.66,54465.00,117981.66,0.00,0.00,0
1,CASH_IN,159445.40,227600.92,387046.31,751211.13,591765.73,0
2,CASH_IN,77739.33,2040.00,79779.33,50056.00,2532.04,0
3,CASH_OUT,135334.88,48786.00,0.00,0.00,135334.88,0
4,CASH_IN,118393.70,50962.00,169355.70,0.00,167607.91,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

Answer here

- In the EDA, we found out that most fraudulent transactions are either `CASH_OUT` or `TRANSFER`. To make this column usable for machine learning models, we can use One Hot Encoding `.get_dummies()`.

In [39]:
encoded_data = pd.get_dummies(data_set, columns=["type"])
encoded_data.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,63516.66,54465.00,117981.66,0.00,0.00,0,True,False,False,False,False
1,159445.40,227600.92,387046.31,751211.13,591765.73,0,True,False,False,False,False
2,77739.33,2040.00,79779.33,50056.00,2532.04,0,True,False,False,False,False
3,135334.88,48786.00,0.00,0.00,135334.88,0,False,True,False,False,False
4,118393.70,50962.00,169355.70,0.00,167607.91,0,True,False,False,False,False


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Answer here

- Since the fraudulent transactions are the minority class, it will cause class imbalance and the model overfit to the majority class, which is the non-fraudulent transactions. To ensure the model learn meaningful patterns, we can use SMOTE to create synthetic minority data.

In [40]:
!pip install imblearn

In [41]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X = encoded_data.drop('isFraud', axis=1)
y = encoded_data['isFraud']

In [42]:
# Apply SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [45]:
# Create a new DataFrame with the resampled data
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['isFraud'] = y_resampled
df_resampled.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,isFraud
0,4991.92,0.00,0.00,0.00,0.00,False,False,False,True,False,0
1,190509.48,2077257.80,2267767.28,1156520.81,966011.33,True,False,False,False,False,0
2,63289.56,0.00,0.00,7564302.19,7627591.75,False,True,False,False,False,0
3,69590.65,0.00,0.00,357084.05,426674.70,False,True,False,False,False,0
4,154130.51,5133566.98,5287697.49,1273335.54,1119205.04,True,False,False,False,False,0


## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

Answer Here

In [47]:
# write out newly transformed dataset to your folder
df_resampled.to_csv('../data/bank_transactions_transformed.csv', index=False)